# Fuzzing in the Large

In this chapter we will discuss steps required to apply your fuzzing to real-world software, assess how well it is performing and ultimately how to scale it to an appropriate amount of resources.

**Prerequisites**

* You should know read and understood the fundamental concepts of fuzzing; for instance, from the chapter ["Fuzzing: Breaking Things with Random Inputs"](Fuzzer.ipynb) and any amount of the more specific chapters following.
* Basic knowledge about Linux and Networking is helpful but not strictly necessary.

## The Real World

When targeting real-world applications, in particular large ones such as browsers, there is a lot of additional factors that influence the outcome of your testing efforts. Some of these factors might seem like trifles to you at first glance, but be aware: In Fuzzing, minor details can have major side-effects and even jeopardize your whole testing.

## Build Configuration and Tools

Testing almost always begins with selecting a build, a certain version and configuration of the software to test. Having control over the build being tested is a huge advantage, especially if you can create your own builds. As we have learned earlier, invariants are highly useful to detect wrong program behavior during fuzzing. Most programs deploy invariant checking in the form of debug-only assertions that need a special type of build (a debug build). Using such a build for fuzzing can yield many more bugs compared to fuzzing a regular build and detect crashes only.

For C/C++ programs, we also previously learned that the use of LLVM Sanitizers, in particular AddressSanitizer, can severely increase our chances to find memory safety violations. The use of sanitizers typically requires you to compile your own build using the LLVM toolchain. For larger projects, this is not necessarily straightforward and might require additional build system work, but the results almost always justify this investment.

Combining what we just discussed, it might seem tempting to just make a debug build with the AddressSanitizer instrumentation and doing so is possible for smaller targets. With larger programs though, the combined slowdown of debug and the additional instrumentation can quickly yield unnecessarily slow builds. As there is little benefit in mixing debug assertions with compile-time instrumentation for memory access checking (they don’t help each other), we can just use debug builds and non-debug sanitizer builds in fuzzing for optimal use of resources.

## Making Software Fuzzing Friendly

There are certain types of behavior for software that can be highly disruptive for fuzzing. One of them is forced aborting/crashing on error conditions that are not considered bugs. While it is sometimes necessary to abort the process, doing so too frequently will make impossible to use fuzzing effectively on the target. Hence the developers should make sure to handle error conditions more gracefully wherever it is possible. It is also helpful to mark forced aborts in such a way that the fuzzing engineers know that these issues are not bugs and they should be ignored. Another typical issue often found is memory leaking. There are situations where fuzzing in particular can cause memory leaks to happen more often than in the regular use case. It is important to identify and fix these cases quickly to avoid slow-downs and aborts due to lack of memory. Most of the time, these conditions are encountered in component-based testing, but also the full product can suffer from these issues. In general, if any of these fixes cannot be enabled by default in the final product, adding specific build options for fuzzing (i.e. a "fuzzing build") can greatly help. One example here is checksums: While required in the final product, checksums can pose problems to fuzzing that could easily be avoided by disabling them altogether in fuzzing builds.

Another aspect is the size of the program: When testing larger programs it can be beneficial to not just test the program as a whole but also test smaller parts of it in isolation. Of course, testing the full program has the advantage that you get comprehensive coverage, in particular of all the interaction between its different components. However, the larger the program under test is, the more likely it is also to suffer from non-determinism (e.g. from threading or other sources of randomness). Performance typically also suffers with growing size and complexity. These factors can be mitigated by allowing parts of the program to be tested standalone, e.g. by breaking larger components out into smaller test shells and/or providing API level entry points for testing.

Yet another way to make software more fuzzing friendly is by adding additional functionality to it that exposes internals in a safe way, e.g. to make edge cases more likely to reproduce. As this approach is very specific for the software being tested, it is hard to give general guidelines. Instead, here is an example from SpiderMonkey, Mozilla’s JavaScript engine, that demonstrates all of the points mentioned in this section:

Even though Mozilla’s JavaScript engine is embedded in the Firefox browser, it can be built at a standalone test shell. It also exposes some C++ API fuzzing targets for testing particularly interesting subcomponents. The JS shell program comes with a command line switch --fuzzing-safe which disables all “unsafe” JavaScript testing APIs that could lead to undesired crashes that are not bugs though. In certain situations, aborting the shell is inevitable, e.g. when an out-of-memory condition happens in a spot where it cannot be handled. In that case, the engine outputs a specific string “[unhandlable oom]” that can be detected and ignored by fuzzing automation. The engine also exposes command line switches and APIs that influence JIT and GC behavior to be more favorable for fuzzing. For example, the “--ion-eager” command line switch compiles all code using the JIT compiler right away while normally, the engine applies a heuristic to determine when code is “hot” and only compiles it then. Using this flag, JIT bugs are much easier to trigger because JIT compilations are more frequent. Another example is GC: While a garbage collect can occur in lots of places, triggering it in one particular spot is hard. Heuristics like memory pressure make garbage collection a particularly volatile experience. On the other hand, many bugs require triggering GC at specific events or at specific locations. In order to allow fuzzers to find these issues more easily, the shell exposes a feature called “gczeal” that performs a configurable type of GC at a specific interval of memory allocations. This feature has exposed tons of GC issues that would otherwise be very hard and unreliable to reproduce.

## Psychological Aspects: The Relationship to Developers

There is not just technical aspects that contribute to success in fuzzing. For every piece of software being tested, there are developers who wrote it. Sometimes, these developers initially perceive you as a nuisance or even a threat to their work: You are not only pointing out potential defects in their code but you are also interrupting their workflow. The latter is particularly impactful, as developers often have tight schedules and sometimes are even pressured to produce new features rather than to fix bugs. There are several reasons though why you should try to alter this perception. First of all, whenever you start fuzzing a sufficiently complex piece of software, there is a number of trivial bugs that have the potential to hold you up, but that have little impact on production deployment. In order to fix these, you might need help from the developers. As we have discussed in the last paragraph, there is also a good opportunity to find additional bugs by adding testing functionality into the software that fuzzers can use to make edge-cases reproducible more easily and reliably. Doing so requires the combined knowledge of the fuzzing engineer and the developer: The fuzzing engineer typically learns what kind of conditions repeatedly cause problems in fuzzing, what areas are under-tested and what the fuzzer can and cannot do. The developer on the other hand knows internals about how the software works and how certain conditions could be met more easily without violating basic constraints of the software. Neither of the two is sufficient to improve testing, this is teamwork. It is hence of utter importance to convince the developers that fuzzing will have a positive impact on their work. A good way to do this is by highlighting how continuous fuzz testing can reduce the number of regressions in production. This type of regression is often particularly stressful for developers as time becomes an essential factor when reacting to production failures.

If larger companies where fuzzing engineers and developers work on the same product, it can also be beneficial to use an embedding strategy where a fuzzing engineer regularly participates in developer meetings. Not only does one learn about new features and internals in such meetings, but more importantly it solidifies the social connection between fuzzing and development. For larger development teams it is also useful to name a specific developer that the fuzzing engineer can regularly interact with and faster than through “official” channels.

## Scaling and Managing Fuzzing Results

Once your fuzzing prototype is up and running and initial problems have been solved, scaling the fuzzing to multiple machines is of course desirable to get more results as quickly as possible. While managing the results of a single machine can be done manually, doing so for a group of machines quickly wastes a lot of time. In particular for larger programs, where not every bug can be fixed immediately, keeping track of known issues and ignoring them is a challenging task. Hence it makes sense to submit all results to a centralized service to manage the results further. Management typically involves a “bucketing” step, where the duplicate issues are grouped together into a single bucket, reporting an issue to a bug tracker and tracking the lifetime of the resulting bug associated with the bucket.

Since fuzzing infrastructure has historically been developed mainly in closed-source corporate environments, there had been few open source tools available for the task. And those that were available were often highly specific for the particular use case that they were built for. In 2014, Mozilla decided to build a generic open source fuzzing management tool called FuzzManager which includes a module called CrashManager for bucketing, reporting and tracking issues on a large scale.

To show you how easy it is to use this kind of automation, we will perform some basic operations around FuzzManager right in the next chapter based on our docker image, containing a simple demo setup.

Whether you settle for one of the existing solutions or come up with your own, keep in mind that automating as much and as early as possible can save a lot of valuable time that can be spent in improving the tools.

## Expect Bugs - In Your Fuzzer

Fuzzing tools are often developed ad hoc and without a proper software development process. Nevertheless, they are just software like any other software and hence subject to the same weaknesses: In particular, no fuzzing tool is bug-free. But how can be sure that our fuzzing tool works as expected? The fact that it finds bugs is a good indicator but that neither guarantees that certain bugs don’t prevent it from finding more, nor does the absence of fuzzing results necessarily mean a defect in the fuzzer. Hence, we should combine this with other checks and as it turns out, there is a very simple, cheap and yet effective indicator available for this purpose: Code coverage. It is common for fuzzing tools to miss certain parts of the code being tested either because of a bug in the fuzzer, missing samples or because the program does not expose the necessary functionality to reach the code. Code coverage counters can furthermore tell us if a fuzzer gets stuck “most of the time” at certain checks and hence doesn’t reach the target area often enough. Getting these metrics in fuzzing is relatively cheap as well, so it is a good choice to start improving the quality of your tools. FuzzManager (the management system mentioned in the last section) even has a module for managing code coverage on a given set of repositories, called CovManager.

Likewise, the next chapter will have simple examples for generating, submitting and inspecting C/C++ coverage in CovManager.